In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [2]:
t=[]
an=[]
p=[]
s=[]
st=[]
ini=[]
L=[]
year_old=[]
bathroom=[]
bhk=[]
project=[]
href1=[]
aminities=[]
loc=input('Enter the City:- ')
url1='https://www.makaan.com/{}-residential-property/buy-property-in-{}-city'.format(loc,loc)
headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
web = requests.get(url1,headers=headers).text
scrap=bs(web)
pages=int(scrap.find('div',class_="search-result-footer").text.split('...')[-1])

for i in range (2):
    url='https://www.makaan.com/{}-residential-property/buy-property-in-{}-city?page={}'.format(loc,loc,i)
    headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
    web1=requests.get(url,headers=headers).text
    scrap1=bs(web1)
    for i in scrap1.find_all('div',class_='cardLayout clearfix'):
        href1.append(i.get('itemid'))
    for i in href1:
        url=i
        headers={'User-Agent':'Mozilla/5.0 (Windows NT 6.3; Win 64 ; x64) Apple WeKit /537.36(KHTML , like Gecko) Chrome/80.0.3987.162 Safari/537.36'}
        web1=requests.get(url,headers=headers).text
        scrap2=bs(web1)
        for i in scrap2.find_all('div',class_='m-scroll-wrap'):
            aminities.append(i.text.split('Not Available')[0])

    for i in scrap1.find_all('div',class_='title-line'):
        t.append(i.text)
    for j in t:
        if 'BHK' in j:
            bhk.append(j.split('BHK')[0])
        elif 'BHK' not in j:
            bhk.append('?')
        for i in t:
            if ' in ' in i:
                project.append(j.split(' in ')[-1])
            else:
                project.append('?')
    for i in scrap1.find_all('div',class_='locWrap'):
        x=i.text
        an.append(x.split(',')[0])
        for i in an:
            L.append(x.split(',')[-1])    
        for i in scrap1.find_all('td',class_='price'):
            a=i.text
            p.append(a)
    for i in scrap1.find_all('td',class_='size'):
        s.append(int(i.text))
    for i in scrap1.find_all('tr',class_="hcol w44"):
        a=i.text
        if 'Under Construction' in a:
            st.append(a.split('Construction')[0]+' Construction')
        else:
            st.append(a.split('Construction')[0])
    for i in scrap1.find_all('ul',class_="listing-details"):
        ini.append(i.text)
        for i in ini:
            if 'years old' in i:
                year_old.append(i.split('years')[0])
            elif 'Possession by'in i:
                if 'Bathrooms' not in i:
                    year_old.append(i.split('New')[0])
                elif 'BathroomsNew' in i:
                    x=i.find('Bathrooms')
                    year_old.append(i.split(i[x-2])[0])
                elif i==None:
                    year_old.append('?')
                else:
                    year_old.append('?')
        for i in ini:
            if 'Bathrooms' in i:
                x=i.find('Bathrooms')
                bathroom.append(i[x-2])
            else:
                bathroom.append('?')
    

Enter the City:- noida


In [3]:
df=pd.read_csv('Noida_Aminities',index_col=[0])

In [17]:
columns=['Project','BHK','Area','City','Price','Size','Status','Old','Bathrooms']
df=pd.DataFrame(zip(project,bhk,an,L,p,s,st,year_old,bathroom),columns=columns)

In [2]:
def check_amenity(amenity, amenities_string):
    return 1 if amenity in amenities_string else 0

amenities_keywords = [
    'Car Parking', 'Multipurpose Room', 'Indoor Games', 'Lift(s)', 'Jogging Track',
    '24 X 7 Security', 'Vaastu Compliant', 'Staff Quarter', 'Rain Water Harvesting',
    'Golf Course', 'Intercom', 'Club House', "Children's play area", 'Full Power Backup',
    'Swimming Pool', 'Gymnasium', 'Landscaped Gardens', 'Sports Facility'
]

for amenity in amenities_keywords:
    df[amenity] = df['amenities'].apply(lambda x: check_amenity(amenity, x))

df.drop('amenities', axis=1, inplace=True)

print(df)


In [18]:
df

,Project,BHK,Area,City,Price,Size,Status,Old,Bathrooms
0,Assotech Windsor Court,3,Sector 78,Noida,1.11 Cr,1645,Ready to move,5 - 6,3
1,Assotech Windsor Court,5,Sector 128,Noida,8.75 Cr,6250,Under Construction,5 - 6,3
2,Assotech Windsor Court,3,Sector 78,Noida,1.11 Cr,1645,Ready to move,Possession by Nov 2023,?
3,Assotech Windsor Court,4,Sector 128,Noida,4.85 Cr,3465,Under Construction,5 - 6,3
4,Assotech Windsor Court,2,Sector 78,Noida,66.82 L,990,Ready to move,Possession by Nov 2023,?
...,...,...,...,...,...,...,...,...,...
395,Amrapali Princely Estate,3,Sector 77,Noida,3.9 Cr,1580,Ready to move,4 - 5,?
396,Amrapali Princely Estate,3,Sector 77,Noida,2.24 Cr,1650,Ready to move,5 - 6,2
397,Amrapali Princely Estate,4,Sector 77,Noida,2.99 Cr,4200,Ready to move,12 - 13,4
398,Amrapali Princely Estate,3,Sector 77,Noida,71.5 L,1970,Ready to move,7 - 8,3


# feature engineering


In [5]:
df['Old'].unique()

array(['4 - 5 ', 'Possession by Nov 2027', '3 - 4 ', '5 - 6 ', '11 - 12 ',
       '6 - 7 ', '14 - 15 ', 'Possession by Sep 2024', '1 - 2 ',
       'Possession by May 2024', '7 ', 'Possession by May '], dtype=object)

In [6]:
df[df['Area']=='Sector 78']

,Project,BHK,Area,City,Price,Size,Status,Old,Bathrooms,aminities,...,Rain Water Harvesting,Golf Course,Intercom,Club House,Children's play area,Full Power Backup,Swimming Pool,Gymnasium,Landscaped Gardens,Sports Facility
0,Assotech Windsor Court,3.0,Sector 78,Noida,1.11 Cr,1645.0,Ready to move,4 - 5,3,ATMLift(s)Jogging TrackFull Power BackupInterc...,...,1,0,1,1,1,1,1,1,1,0
2,Assotech Windsor Court,3.0,Sector 78,Noida,1.11 Cr,1645.0,Ready to move,Possession by Nov 2027,?,ATMLift(s)Jogging TrackFull Power BackupInterc...,...,1,0,1,1,1,1,1,1,1,0
4,Assotech Windsor Court,2.0,Sector 78,Noida,66.82 L,990.0,Ready to move,Possession by Nov 2027,?,ATMLift(s)Jogging TrackFull Power BackupInterc...,...,1,0,1,1,1,1,1,1,1,0
20,Mahagun Medalleo,3.0,Sector 78,Noida,1.11 Cr,1645.0,Ready to move,Possession by Nov 2027,3,Swimming PoolLift(s)CafeteriaIntercomIndoor Ga...,...,0,0,1,1,1,0,1,1,0,1
22,Mahagun Medalleo,3.0,Sector 78,Noida,1.11 Cr,1645.0,Ready to move,Possession by Nov 2027,?,ATMLift(s)Jogging TrackFull Power BackupInterc...,...,1,0,1,1,1,1,1,1,1,0
24,Mahagun Medalleo,2.0,Sector 78,Noida,66.82 L,990.0,Ready to move,Possession by Nov 2027,?,ATMLift(s)Jogging TrackFull Power BackupInterc...,...,1,0,1,1,1,1,1,1,1,0
53,?,3.0,Sector 78,Noida,59 L,2800.0,Ready to move,5 - 6,2,Club HouseLift(s)GymnasiumFull Power BackupInt...,...,0,0,1,1,1,1,1,1,0,1
78,Mahagun Medalleo,4.0,Sector 78,Noida,2.24 Cr,2105.0,Ready to move,4 - 5,3,Indoor GamesShopping MallJogging TrackLift(s)M...,...,1,0,1,1,1,1,1,1,0,1
120,AVP AVS Orchard,3.0,Sector 78,Noida,1.11 Cr,990.0,Ready to move,6 - 7,3,GymnasiumLift(s)IntercomSports FacilityCar Par...,...,0,0,1,1,1,1,0,1,0,1
121,AVP AVS Orchard,4.0,Sector 78,Noida,3.83 Cr,1390.0,Ready to move,14 - 15,?,Gas connectionAC,...,0,0,0,0,0,0,0,0,0,0


In [238]:
def area_other(series,series2):
    for i,j in zip(series,series2):
        if i<10:
            df['Area'].replace(j,'Other',inplace=True)
        
    

In [239]:
area_other(df['Area'].value_counts(),df['Area'].value_counts().index)

# Price,Size astype INTEGER 

In [22]:
Price_Lakh=[]
for i in df['Price']:
    if 'Cr' in i:
        Price_Lakh.append(float(i.split('Cr')[0].strip())*100)
    elif 'L' in i:
        Price_Lakh.append(float(i.split('L')[0].strip()))


In [23]:
df['Price']=Price_Lakh

In [9]:
df['Size']=df['Size'].astype('float')

In [10]:
df['BHK'].replace(['?'],[None],inplace=True)

In [11]:
df.dropna(inplace=True)

In [12]:
df['BHK']=df['BHK'].astype('int')

# Bathrooms

In [33]:
na_bath=df[df['Bathrooms']=='?']

In [34]:
num_bath=df[df['Bathrooms']!='?']

In [35]:
num_bath['BHK'].value_counts()

3     219
2     146
4     134
5      26
6      14
10      4
1       2
Name: BHK, dtype: int64

In [41]:
num_bath['Bathrooms']=num_bath['Bathrooms'].astype(int)

C:\Users\gusai\AppData\Local\Temp\ipykernel_3068\3986576913.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_bath['Bathrooms']=num_bath['Bathrooms'].astype(int)


In [44]:
bath_mean=int(num_bath['Bathrooms'].mean())

In [45]:
df['Bathrooms'].replace('?',int(bath_mean),inplace=True)

In [46]:
df['Bathrooms'].value_counts()

2    294
3    259
2    205
1     47
5     33
7      1
Name: Bathrooms, dtype: int64

In [47]:
df['Bathrooms']=df['Bathrooms'].astype('int')

# Status,OLD

In [48]:
def old_m(series):
    for i in series:
        if 'Possession' in i:
            df['Old'].replace(i,-1,inplace=True)
        elif '-' in i:
            df['Old'].replace(i,i.split('-')[-1],inplace=True)
        
            

In [49]:
old=[]
for i in df['Old']:
    if '-' in i:
        old.append(i.split('-')[-1].strip())
    elif 'Possession' in i:
        old.append(-1)
    else:
        old.append(i)
    

In [50]:
df['Old']=old

In [51]:
df['Old']=df['Old'].astype('int')

In [52]:
S=df['Status'].value_counts()

In [53]:
for i in S.index:
    if 'Resale' in i:
        df['Status'].replace(i,'Ready to move',inplace=True)
    

In [54]:
df.isnull().sum()

Project                  0
BHK                      0
Area                     0
City                     0
Price                    0
Size                     0
Status                   0
Old                      0
Bathrooms                0
aminities                0
Price_Lakh               0
Car Parking              0
Multipurpose Room        0
Indoor Games             0
Lift(s)                  0
Jogging Track            0
24 X 7 Security          0
Vaastu Compliant         0
Staff Quarter            0
Rain Water Harvesting    0
Golf Course              0
Intercom                 0
Club House               0
Children's play area     0
Full Power Backup        0
Swimming Pool            0
Gymnasium                0
Landscaped Gardens       0
Sports Facility          0
dtype: int64

In [58]:
df.drop('aminities',axis=1,inplace=True)

In [60]:
df['Status'].unique()

array(['Ready to move', 'Under  Construction'], dtype=object)

In [70]:
df['Size'].value_counts()

1050.0    95
650.0     32
950.0     20
3720.0    16
2500.0    15
          ..
1225.0     1
3047.0     1
6300.0     1
3168.0     1
1550.0     1
Name: Size, Length: 229, dtype: int64